In [1]:
import gzip
import os
import re
import numpy as np
import wave
from six.moves import urllib
import tflearn

hdf5 not supported (please install/reinstall h5py)


In [2]:
os.chdir('/home/cc/Data')
import speech_data as data
pcm_path = 'data/spoken_numbers_pcm/'

In [3]:
speakers = []
for f1 in os.listdir(pcm_path):
    speakers.append(f1)
number_classes=len(speakers)
print('speakers',speakers)

('speakers', ['Speaker9', 'Speaker4', 'Speaker2', 'Speaker3', 'Speaker1', 'Speaker5', 'Speaker7', 'Speaker0', 'Speaker8', 'Speaker6'])


In [4]:
# get the number of classes...aka len(speakers)
number_classes = len(speakers)
print('speakers', speakers)

('speakers', ['Speaker9', 'Speaker4', 'Speaker2', 'Speaker3', 'Speaker1', 'Speaker5', 'Speaker7', 'Speaker0', 'Speaker8', 'Speaker6'])


In [5]:

WORD_WAVs="spoken_words"
batch=data.wave_batch_generator(source=WORD_WAVs,target=data.Target.speaker, speakers=speakers, batch_size=1000)
X,Y=next(batch)

In [6]:
# Classification
tflearn.init_graph(num_cores=8, gpu_memory_fraction=0.5)

net = tflearn.input_data(shape=[None, 8192]) #Two wave chunks
net = tflearn.fully_connected(net, 64)
net = tflearn.dropout(net, 0.5)
net = tflearn.fully_connected(net, number_classes, activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

In [7]:
# training step 
model = tflearn.DNN(net)
model.fit(X, Y, n_epoch=5000, show_metric=True, snapshot_step=100)

Training Step: 80000  | total loss: 0.11438
| Adam | epoch: 5000 | loss: 0.11438 - acc: 0.9746 -- iter: 1000/1000
Training Step: 80000  | total loss: 0.11438
| Adam | epoch: 5000 | loss: 0.11438 - acc: 0.9746 -- iter: 1000/1000
--


In [8]:
demo_file = "Speaker2_wav25.wav"
demo=data.load_wav_file(demo_file)
result=model.predict([demo])
result=data.one_hot_to_item(result,speakers)
print("predicted speaker for %s : result = %s "%(demo_file,result))

predicted speaker for Speaker2_wav25.wav : result = Speaker2 
